In [ ]:
# import some basic functions
import os
import numpy as np
import re
from skimage import io
import math
import pandas as pd

In [ ]:
def coefficient_of_variances(gc_mask: np.ndarray, img: np.ndarray, cell_id: str, measured_parameters: List):
    """
    Parameters:
    -----------
    gc_mask: 3D array
        the gc mask has been filled holes
    img: nD array
        3 channel stacked image
    radius: int
        the dilation radius
    cell_id: str
        the id of the cell: in the form of: 'experiment_set'_'cell folder'
    measured_parameters: List
        list of parameters: ["cell_id", "cv_r", "cv_g", "cv_b", "qcd_r", "qcd_g", "qcd_b"]
    
    Returns:
    --------
    cv: list
        list of cv of each channel as the order: r,g,b
    qcd: list
        list of qcd of each channel as the order: r,g,b
    """
    ################################
    # process mask                 #
    ################################
    mask_size = np.count_nonzero(gc_mask)*(0.2*0.08*0.08)

    # extract gray value of each channel from dilated mask
    raw_red_in_mask = img[...,0][gc_mask>0]
    raw_green_in_mask = img[...,1][gc_mask>0]
    raw_blue_in_mask = img[...,2][gc_mask>0]

    ################################
    # process mask   done          #
    ################################
    # create a pd_dataframe to store measurements
    variables: Dict[str, Any] = {}
    for key in measured_parameters:
        variables[key] = None
    df = pd.DataFrame(variables, index=[0]) 

    # measure coefficient of variance
    cv_r = round(np.std(raw_red_in_mask)/np.mean(raw_red_in_mask),3)
    cv_g = round(np.std(raw_green_in_mask)/np.mean(raw_green_in_mask),3)
    cv_b = round(np.std(raw_blue_in_mask)/np.mean(raw_blue_in_mask),3)

    # measure quartile coefficient of dispersion, Q3-Q1/Q3+Q1, descriptive measurement of dispersion,less sensitive to outliers
    qcd_r = round((np.quantile(raw_red_in_mask,0.75)-np.quantile(raw_red_in_mask,0.25))/(np.quantile(raw_red_in_mask,0.75)+np.quantile(raw_red_in_mask,0.25)),3)
    qcd_g = round((np.quantile(raw_green_in_mask,0.75)-np.quantile(raw_green_in_mask,0.25))/(np.quantile(raw_green_in_mask,0.75)+np.quantile(raw_green_in_mask,0.25)),3)
    qcd_b = round((np.quantile(raw_blue_in_mask,0.75)-np.quantile(raw_blue_in_mask,0.25))/(np.quantile(raw_blue_in_mask,0.75)+np.quantile(raw_blue_in_mask,0.25)),3)

    # store values in the pd_dataframe
    df.loc[0,df.columns] = pd.Series([cell_id,cv_r,cv_g,cv_b,qcd_r,qcd_g,qcd_b],index=df.columns)

    return df

In [ ]:
# measure cv and generate dataframe
cv_df = coefficient_of_variances(gc_mask, raw_bg_subtracted, cell_id=cell_id,measured_parameters = ["cell_id", "cv_r", "cv_g", "cv_b", "qcd_r", "qcd_g", "qcd_b"])